### 1. Import all the necessary libraries

In [6]:
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.optim as optim
import category_encoders as ce
%matplotlib inline
import torch.nn as nn
import torch.nn.functional as F

### 2. Loading the Dataset

In [7]:
df_test = pd.read_csv("../test/test_public.csv")
df_test.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA
0,T1,B,NaN,15.0,20000542,1408039037,A,False
1,T2,B,NaN,57.0,20000108,1408038611,A,False
2,T3,B,NaN,15.0,20000370,1408038568,A,False
3,T4,B,NaN,53.0,20000492,1408039090,A,False
4,T5,B,NaN,18.0,20000621,1408039177,A,False


In [11]:
# net A
def _weight_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight)
    elif isinstance(m, nn.BatchNorm1d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)

class Net_A(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(24, 256)
        self.fc2 = nn.Linear(256, 256)
        self.fc3 = nn.Linear(256, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)  # New layer: fc5 -> fc6
        self.fc6 = nn.Linear(64, 32)   # New layer: fc6 -> fc7
        self.fc7 = nn.Linear(32, 1)    # New layer: fc7 -> output
        self.apply(_weight_init)
        self.apply(_weight_init) # 初始化参数
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = self.fc7(x)
        return x

net_A = Net_A()
net_A.load_state_dict(torch.load('../model/modelA.pth'))

<All keys matched successfully>

In [12]:
# net B
class Net_B(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(95, 256)
        self.fc2 = nn.Linear(256,256)
        self.fc3 = nn.Linear(256,256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 1)
        self.apply(_weight_init) # 初始化参数
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

net_B = Net_B()
net_B.load_state_dict(torch.load('../model/modelB.pth'))

<All keys matched successfully>

In [13]:
# net C
class Net_C(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(17, 512)
        self.fc2 = nn.Linear(512,512)
        self.fc3 = nn.Linear(512,512)
        self.fc4 = nn.Linear(512, 256)
        self.fc5 = nn.Linear(256, 1)
        self.apply(_weight_init) # 初始化参数
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

net_C = Net_C()
net_C.load_state_dict(torch.load('../model/modelC.pth'))

<All keys matched successfully>

In [14]:




# df_test = df_test.drop(columns=[ 'TAXI_ID', 'TIMESTAMP', 'MISSING_DATA', 'YR',
#                                 'DAY'])

# df_test['MON'] = df_test['MON'].astype(str)
# df_test['WK'] = df_test['WK'].astype(str)
# df_test['HR'] = df_test['HR'].astype(str)

# dummy_test = pd.get_dummies(df_test[['MON', 'WK', 'HR']])
# df_test = pd.concat([df_test, dummy_test], axis=1)

# # test_wk = df_test[df_test["WK"] == 7]
# # print(test_wk)
# df_test = df_test.drop(columns=['MON', 'WK', 'HR'])

# df_test['MON_1'] = 0
# df_test['MON_2'] = 0
# df_test['MON_3'] = 0
# df_test['MON_4'] = 0
# df_test['MON_5'] = 0
# df_test['MON_6'] = 0
# df_test['MON_7'] = 0

# df_test['WK_2'] = 0
# df_test['WK_4'] = 0

# df_test['HR_0'] = 0
# df_test['HR_1'] = 0
# df_test['HR_4'] = 0
# df_test['HR_5'] = 0
# df_test['HR_9'] = 0
# df_test['HR_10'] = 0
# df_test['HR_18'] = 0
# df_test['HR_19'] = 0
# df_test['HR_20'] = 0
# df_test['HR_21'] = 0
# df_test['HR_22'] = 0
# df_test['HR_23'] = 0


# for index, row in df_test.iterrows():
#     if row["CALL_TYPE"] == "A":
#         print(row["TRIP_ID"])

# for index, row in df_test.iterrows():
#     if row["CALL_TYPE"] == "A":
#         df_test['Unique_TAXI_ID'].iloc[index] = (df_test['Unique_TAXI_ID'].iloc[index] - 333.4006168270417) / 209.67678198094498
#         df_test['ORIGIN_CALL'].iloc[index] = (df_test['ORIGIN_CALL'].iloc[index] - 24490.363017792035) / 19624.29004302366
#     elif row["CALL_TYPE"] == "B":
#         df_test['Unique_TAXI_ID'].iloc[index] = (df_test['Unique_TAXI_ID'].iloc[index] - 346.12949952304933) / 210.39338583617038
#         df_test['ORIGIN_STAND'].iloc[index] = (df_test['ORIGIN_STAND'].iloc[index] - 30.2782090247832) / 17.743860394485583
#     elif row["CALL_TYPE"] == "C":
#         df_test['Unique_TAXI_ID'].iloc[index] = (df_test['Unique_TAXI_ID'].iloc[index] - 362.4130874078395) / 212.65140390214862

# # cols = list(df_test.columns.values)
# # print(cols)

# print(df_test)

In [15]:
from datetime import datetime

def parse_time(x):
  # We are using python's builtin datetime library
  # https://docs.python.org/3/library/datetime.html#datetime.date.fromtimestamp

  # Each x is essentially a 1 row, 1 column pandas Series
  dt = datetime.fromtimestamp(x["TIMESTAMP"])
  return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

def parse_TAXI_ID(x):
    return (x % pow(10,3)) 

In [16]:
df_test = pd.read_csv("../test/test_public.csv")

df_test[["YR", "MON", "DAY", "HR", "WK"]] = df_test[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

df_test["Unique_TAXI_ID"] = df_test["TAXI_ID"].apply(parse_TAXI_ID)

df_test

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,YR,MON,DAY,HR,WK,Unique_TAXI_ID
0,T1,B,NaN,15.0,20000542,1408039037,A,False,2014,8,14,17,3,542
1,T2,B,NaN,57.0,20000108,1408038611,A,False,2014,8,14,17,3,108
2,T3,B,NaN,15.0,20000370,1408038568,A,False,2014,8,14,17,3,370
3,T4,B,NaN,53.0,20000492,1408039090,A,False,2014,8,14,17,3,492
4,T5,B,NaN,18.0,20000621,1408039177,A,False,2014,8,14,17,3,621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315,T323,A,70885.0,NaN,20000430,1419171485,A,False,2014,12,21,14,6,430
316,T324,B,NaN,53.0,20000020,1419170802,A,False,2014,12,21,14,6,20
317,T325,C,NaN,NaN,20000207,1419172121,A,False,2014,12,21,14,6,207
318,T326,A,76232.0,NaN,20000667,1419171980,A,False,2014,12,21,14,6,667


In [17]:
def eval(call_type, test_set, net):
    
    if call_type == "A":
        
        required_cols = ['ORIGIN_CALL_0', 'ORIGIN_CALL_1', 'ORIGIN_CALL_2', 
                         'ORIGIN_CALL_3', 'ORIGIN_CALL_4', 'ORIGIN_CALL_5', 
                         'ORIGIN_CALL_6', 'ORIGIN_CALL_7', 'ORIGIN_CALL_8', 
                         'ORIGIN_CALL_9', 'ORIGIN_CALL_10', 'ORIGIN_CALL_11', 
                         'ORIGIN_CALL_12', 'ORIGIN_CALL_13', 'ORIGIN_CALL_14', 
                         'ORIGIN_CALL_15', 'HR_0', 'HR_1', 'HR_2', 'HR_3', 'HR_4', 
                         'WK_0', 'WK_1', 'WK_2']
        
        all_features = pd.DataFrame(columns=required_cols)
        df_tr_test = test_set
        
#         print(df_tr_test)
        
        all_features_test = df_tr_test[[]]

        # Create an instance of BinaryEncoder
        binary_encoder_origin_test = ce.BinaryEncoder(cols=['ORIGIN_CALL'])
        binary_encoder_hr_test = ce.BinaryEncoder(cols=['HR'])
        binary_encoder_wk_test = ce.BinaryEncoder(cols=['WK'])


        # Apply binary encoding to the 'ORIGIN_CALL' column
        encoded_data_origin_test = binary_encoder_origin_test.fit_transform(df_tr_test['ORIGIN_CALL'])
        encoded_data_hr_test = binary_encoder_hr_test.fit_transform(df_tr_test['HR'])
        encoded_data_wk_test = binary_encoder_wk_test.fit_transform(df_tr_test['WK'])

        # Concatenate the encoded data with the original DataFrame
        all_features_test = pd.concat([all_features_test, encoded_data_origin_test], axis=1)
        all_features_test = pd.concat([all_features_test, encoded_data_hr_test], axis=1)
        all_features_test = pd.concat([all_features_test, encoded_data_wk_test], axis=1)

        missing_columns = set(all_features.columns)-set(all_features_test.columns)

        for column in missing_columns:
            all_features_test[column] = 0

        prediction = net(torch.tensor(all_features_test.values, dtype=torch.float))
        
        return prediction
    
    elif call_type == "B":
        
        required_cols = ['ORIGIN_STAND_1.0', 'ORIGIN_STAND_10.0', 'ORIGIN_STAND_11.0', 'ORIGIN_STAND_12.0', 
                         'ORIGIN_STAND_13.0', 'ORIGIN_STAND_14.0', 'ORIGIN_STAND_15.0', 'ORIGIN_STAND_16.0', 
                         'ORIGIN_STAND_17.0', 'ORIGIN_STAND_18.0', 'ORIGIN_STAND_19.0', 'ORIGIN_STAND_2.0', 
                         'ORIGIN_STAND_20.0', 'ORIGIN_STAND_21.0', 'ORIGIN_STAND_22.0', 'ORIGIN_STAND_23.0', 
                         'ORIGIN_STAND_24.0', 'ORIGIN_STAND_25.0', 'ORIGIN_STAND_26.0', 'ORIGIN_STAND_27.0', 
                         'ORIGIN_STAND_28.0', 'ORIGIN_STAND_29.0', 'ORIGIN_STAND_3.0', 'ORIGIN_STAND_30.0', 
                         'ORIGIN_STAND_31.0', 'ORIGIN_STAND_32.0', 'ORIGIN_STAND_33.0', 'ORIGIN_STAND_34.0', 
                         'ORIGIN_STAND_35.0', 'ORIGIN_STAND_36.0', 'ORIGIN_STAND_37.0', 'ORIGIN_STAND_38.0', 
                         'ORIGIN_STAND_39.0', 'ORIGIN_STAND_4.0', 'ORIGIN_STAND_40.0', 'ORIGIN_STAND_41.0', 
                         'ORIGIN_STAND_42.0', 'ORIGIN_STAND_43.0', 'ORIGIN_STAND_44.0', 'ORIGIN_STAND_45.0', 
                         'ORIGIN_STAND_46.0', 'ORIGIN_STAND_47.0', 'ORIGIN_STAND_48.0', 'ORIGIN_STAND_49.0', 
                         'ORIGIN_STAND_5.0', 'ORIGIN_STAND_50.0', 'ORIGIN_STAND_51.0', 'ORIGIN_STAND_52.0', 
                         'ORIGIN_STAND_53.0', 'ORIGIN_STAND_54.0', 'ORIGIN_STAND_55.0', 'ORIGIN_STAND_56.0', 
                         'ORIGIN_STAND_57.0', 'ORIGIN_STAND_58.0', 'ORIGIN_STAND_59.0', 'ORIGIN_STAND_6.0', 
                         'ORIGIN_STAND_60.0', 'ORIGIN_STAND_61.0', 'ORIGIN_STAND_62.0', 'ORIGIN_STAND_63.0', 
                         'ORIGIN_STAND_7.0', 'ORIGIN_STAND_8.0', 'ORIGIN_STAND_9.0', 'ORIGIN_STAND_nan', 'WK_0', 
                         'WK_1', 'WK_2', 'WK_3', 'WK_4', 'WK_5', 'WK_6', 'HR_0', 'HR_1', 'HR_10', 'HR_11', 'HR_12', 
                         'HR_13', 'HR_14', 'HR_15', 'HR_16', 'HR_17', 'HR_18', 'HR_19', 'HR_2', 'HR_20', 'HR_21', 
                         'HR_22', 'HR_23', 'HR_3', 'HR_4', 'HR_5', 'HR_6', 'HR_7', 'HR_8', 'HR_9']
        
        all_features = pd.DataFrame(columns=required_cols)
        df_tr_test = test_set
        
        all_features_test = df_tr_test[["ORIGIN_STAND","WK","HR"]]

        all_features_test["ORIGIN_STAND"] = all_features_test["ORIGIN_STAND"].astype(str)
        all_features_test['WK'] = all_features_test['WK'].astype(str)
        all_features_test['HR'] = all_features_test['HR'].astype(str)

        all_features_test = pd.get_dummies(all_features_test)

        missing_columns = set(all_features.columns)-set(all_features_test.columns)

        for column in missing_columns:
            all_features_test[column] = 0
        
        prediction = net(torch.tensor(all_features_test.values, dtype=torch.float))
        
        return prediction
    
    elif call_type == "C":
        required_cols = ['Unique_TAXI_ID_0', 'Unique_TAXI_ID_1', 'Unique_TAXI_ID_2', 'Unique_TAXI_ID_3', 
                         'Unique_TAXI_ID_4', 'Unique_TAXI_ID_5', 'Unique_TAXI_ID_6', 'Unique_TAXI_ID_7', 
                         'Unique_TAXI_ID_8', 'HR_0', 'HR_1', 'HR_2', 'HR_3', 'HR_4', 'WK_0', 'WK_1', 'WK_2']
        
        all_features = pd.DataFrame(columns=required_cols)
        df_tr_test = test_set
    
        all_features_test = df_tr_test[[]]

        # Create an instance of BinaryEncoder
        binary_encoder_id_test = ce.BinaryEncoder(cols=['Unique_TAXI_ID'])
        binary_encoder_hr_test = ce.BinaryEncoder(cols=['HR'])
        binary_encoder_wk_test = ce.BinaryEncoder(cols=['WK'])


        # Apply binary encoding to the 'ORIGIN_CALL' column
        encoded_data_id_test = binary_encoder_id_test.fit_transform(df_tr_test['Unique_TAXI_ID'])
        encoded_data_hr_test = binary_encoder_hr_test.fit_transform(df_tr_test['HR'])
        encoded_data_wk_test = binary_encoder_wk_test.fit_transform(df_tr_test['WK'])

        # Concatenate the encoded data with the original DataFrame
        all_features_test = pd.concat([all_features_test, encoded_data_id_test], axis=1)
        all_features_test = pd.concat([all_features_test, encoded_data_hr_test], axis=1)
        all_features_test = pd.concat([all_features_test, encoded_data_wk_test], axis=1)

        missing_columns = set(all_features.columns)-set(all_features_test.columns)

        for column in missing_columns:
            all_features_test[column] = 0

        prediction = net(torch.tensor(all_features_test.values, dtype=torch.float))
        
        return prediction
    
    else:
        raise Exception("Illegal Call Type!!!")

In [18]:
prediction_dict = {}
df_test_typeA = df_test[df_test["CALL_TYPE"]=="A"]
df_test_typeB = df_test[df_test["CALL_TYPE"]=="B"]
df_test_typeC = df_test[df_test["CALL_TYPE"]=="C"]

predictions_A = eval(call_type="A", test_set=df_test_typeA, net=net_A)
predictions_B = eval(call_type="B", test_set=df_test_typeB, net=net_B)
predictions_C = eval(call_type="C", test_set=df_test_typeC, net=net_C)

for idx in range(len(df_test_typeA)):
    prediction_dict[df_test_typeA["TRIP_ID"].iloc[idx]] = predictions_A[idx]
    
for idx in range(len(df_test_typeB)):
    prediction_dict[df_test_typeB["TRIP_ID"].iloc[idx]] = predictions_B[idx]
    
for idx in range(len(df_test_typeC)):
    prediction_dict[df_test_typeC["TRIP_ID"].iloc[idx]] = predictions_C[idx]

prediction_dict

/tmp/ipykernel_109411/533806469.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_features_test["ORIGIN_STAND"] = all_features_test["ORIGIN_STAND"].astype(str)
/tmp/ipykernel_109411/533806469.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_features_test['WK'] = all_features_test['WK'].astype(str)
/tmp/ipykernel_109411/533806469.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

{'T6': tensor([487.3210], grad_fn=<SelectBackward0>),
 'T8': tensor([367.9285], grad_fn=<SelectBackward0>),
 'T22': tensor([539.7828], grad_fn=<SelectBackward0>),
 'T23': tensor([482.3542], grad_fn=<SelectBackward0>),
 'T37': tensor([596.9558], grad_fn=<SelectBackward0>),
 'T60': tensor([664.8036], grad_fn=<SelectBackward0>),
 'T64': tensor([596.9558], grad_fn=<SelectBackward0>),
 'T68': tensor([593.5859], grad_fn=<SelectBackward0>),
 'T81': tensor([344.1285], grad_fn=<SelectBackward0>),
 'T83': tensor([344.1285], grad_fn=<SelectBackward0>),
 'T84': tensor([464.0434], grad_fn=<SelectBackward0>),
 'T86': tensor([499.7385], grad_fn=<SelectBackward0>),
 'T91': tensor([578.1052], grad_fn=<SelectBackward0>),
 'T92': tensor([663.6332], grad_fn=<SelectBackward0>),
 'T95': tensor([663.6332], grad_fn=<SelectBackward0>),
 'T99': tensor([493.1104], grad_fn=<SelectBackward0>),
 'T100': tensor([543.7289], grad_fn=<SelectBackward0>),
 'T101': tensor([479.7131], grad_fn=<SelectBackward0>),
 'T111': t

In [19]:
# print(df_test)

df_sample = pd.read_csv("../test/sampleSubmission.csv")

for index, row in df_test.iterrows():
    predict = prediction_dict[row["TRIP_ID"]]
    df_sample["TRAVEL_TIME"].iloc[index] = predict.detach().squeeze().numpy()
#     if row["CALL_TYPE"] == "A":
#         predict = eval(call_type="A", test_set=row, net = net_A)
#         print(row["TRIP_ID"] ,predict.detach().squeeze().numpy())
#         predict_features = row[['Unique_TAXI_ID', 'ORIGIN_CALL', 'HR_0', 'HR_1', 'HR_10', 'HR_11', 'HR_12', 'HR_13', 'HR_14', 'HR_15', 'HR_16', 'HR_17', 'HR_18', 'HR_19', 'HR_2', 'HR_20', 'HR_21', 'HR_22', 'HR_23', 'HR_3', 'HR_4', 'HR_5', 'HR_6', 'HR_7', 'HR_8', 'HR_9']]
#         test_features = torch.tensor(predict_features, dtype=torch.float)
#         predict = eval(call_type="A", test_set=test_features, net = net_A)net_A(test_features)
#         print(row["TRIP_ID"] ,predict.detach().squeeze().numpy())
#         df_sample["TRAVEL_TIME"].iloc[index] = predict.detach().squeeze().numpy()
#     elif row["CALL_TYPE"] == "B":
#         predict_features = row[['Unique_TAXI_ID', 'ORIGIN_STAND', 'MON_1', 'MON_10', 
#                                 'MON_11', 'MON_12', 'MON_2', 'MON_3', 'MON_4', 'MON_5', 
#                                 'MON_6', 'MON_7', 'MON_8', 'MON_9', 'WK_0', 'WK_1', 'WK_2', 
#                                 'WK_3', 'WK_4', 'WK_5', 'WK_6', 'HR_0', 'HR_1', 'HR_10', 
#                                 'HR_11', 'HR_12', 'HR_13', 'HR_14', 'HR_15', 'HR_16', 
#                                 'HR_17', 'HR_18', 'HR_19', 'HR_2', 'HR_20', 'HR_21', 
#                                 'HR_22', 'HR_23', 'HR_3', 'HR_4', 'HR_5', 'HR_6', 
#                                 'HR_7', 'HR_8', 'HR_9']]
#         test_features = torch.tensor(predict_features, dtype=torch.float)
#         predict = net_B(test_features)
#         print(row["TRIP_ID"] ,predict.detach().squeeze().numpy())
#         df_sample["TRAVEL_TIME"].iloc[index] = predict.detach().squeeze().numpy()
#     elif row["CALL_TYPE"] == "C":
#         predict_features = row[['Unique_TAXI_ID', 'MON_1', 'MON_10', 'MON_11', 'MON_12', 
#                                 'MON_2', 'MON_3', 'MON_4', 'MON_5', 'MON_6', 'MON_7', 
#                                 'MON_8', 'MON_9', 'WK_0', 'WK_1', 'WK_2', 'WK_3', 'WK_4', 
#                                 'WK_5', 'WK_6', 'HR_0', 'HR_1', 'HR_10', 'HR_11', 'HR_12', 
#                                 'HR_13', 'HR_14', 'HR_15', 'HR_16', 'HR_17', 'HR_18', 
#                                 'HR_19', 'HR_2', 'HR_20', 'HR_21', 'HR_22', 'HR_23', 
#                                 'HR_3', 'HR_4', 'HR_5', 'HR_6', 'HR_7', 'HR_8', 'HR_9']]
#         test_features = torch.tensor(predict_features, dtype=torch.float)
#         predict = net_C(test_features)
#         print(row["TRIP_ID"] ,predict.detach().squeeze().numpy())
#         df_sample["TRAVEL_TIME"].iloc[index] = predict.detach().squeeze().numpy()

# test_features = torch.tensor(df_test.values, dtype=torch.float)

# predict = net(val_features)
# predict = predict.detach().squeeze().numpy()
# print(predict[:1000])

df_sample.to_csv("my_pred.csv", index=None)

/tmp/ipykernel_109411/4131001495.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sample["TRAVEL_TIME"].iloc[index] = predict.detach().squeeze().numpy()


In [20]:
# Over every single 
def polyline_to_trip_duration(polyline):
  return max(polyline.count("[") - 1, 0) * 15

# This code creates a new column, "LEN", in our dataframe. The value is
# the (polyline_length - 1) * 15
df_tr_test = pd.read_csv("../test/test.csv")
df_tr_test.head()

df_tr_test["LEN"] = df_tr_test["POLYLINE"].apply(polyline_to_trip_duration)
df_tr_test[["YR", "MON", "DAY", "HR", "WK"]] = df_tr_test[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")

In [21]:
predictttt = df_sample["TRAVEL_TIME"]
reallllll = df_tr_test["LEN"]

In [22]:
import math
def custom_score(y_true, y_pred):
#     rmse = mean_squared_error(np.log1p(y_true), np.log1p(y_pred), squared=False)
#     return rmse
    return math.sqrt(np.mean((np.array(y_pred)-np.array(y_true))*(np.array(y_pred)-np.array(y_true))))

print("The real loss is :")
custom_score(reallllll,predictttt)

The real loss is :


980.5013400045522

In [23]:
df_tr_test = pd.read_csv("../test/test.csv")
df_tr_test = df_tr_test.drop(columns=['POLYLINE'])

In [52]:
df_test = pd.read_csv("../test/test_public.csv")

In [54]:
if df_tr_test.equals(df_test):
    print("YEAH")

YEAH
